### Importing modules

In [277]:
import numpy as np
import matplotlib.pyplot as plt
import os
from enum import Enum
from itertools import permutations, combinations, product
from copy import copy
from icecream import ic
from math import atan
import logging
from functools import cmp_to_key
from tabulate import tabulate
import random

### Input format
The first line is of the form
```
num_agents,nego_est_o,nego_dl_o,tasks_est_o,tasks_dl_o
```
Here,
- `num_agents` is the number of agents
- `nego_est_o` is the outside earliest starting time of the negotiation
- `nego_dl_o` is the outside deadline of the negotiation
- `tasks_est_o` is the outside earliest starting time of the task execution
- `tasks_dl_o` is the outside deadline of the task execution

It is followed by `issue_num` lines, of which line $i$ (zero-indexed) of the input .csv file pertains to issue $i$, and is a tuple of the form `(agent, parent_issue, [pretasks], children_finish, issue_type, other_issue, pbs, reward, decommitment_penalty_rate, est_o, dl_o, duration)`, where

- `agent` is the agent ID of that agent which is supposed to do this task
- `parent_issue` of an incoming issue is **not** the same as that of its `other_issue`
- `[pretasks]` = Set of explicit pretasks (i.e., doesn't include parent-child relationships). '' if there is no pretask. Separated by ';' otherwise
- `children_finish` = 0 for "and", 1 for "or", '' for no chlidren. It may be 0 or 1 even if it has no children; however, '' indicates that we know for sure that there is no child.
- `issue_type` = 1 for local tasks (that are not incoming), 2 for outgoing tasks, 3 for incoming tasks
- `other_issue` is the issue ID of the issue linked to this issue (defined only when issue_type is outgoing or incoming)
- `pbs` is the base probability of success
- `reward` is the completion reward of the task, defined only for incoming issues
- `decommitment_penalty_rate` is the decommitment penalty rate, defined only for incoming issues. It is implicit that it is the same value for the corresponding outgoing issue. Meaningless for other local tasks.
- `est_o` is the outside earliest starting time of this task. The default is the overall `est_o`. For outgoing issues, it is undefined and taken to be that of the `other_issue`'s.
- `dl_o` is the outside deadline of this task. The default is the overall `dl_o`. For outgoing issues, it is undefined and taken to be that of the `other_issue`'s.
- `duration` is the number of timesteps needed to execute this task. Undefined for outgoing tasks.

### Defining classes to store the data

In [278]:
class issue_type(Enum):
    NONE = 0
    LOCAL = 1
    OUTGOING = 2
    INCOMING = 3


class issue:
    def __init__(self, issue_num: int, line: str, tasks_est_o, tasks_dl_o) -> None:
        self.ID = issue_num
        line = line.split(",")
        self.agent = int(line[0])
        self.parent = int(line[1])
        try:
            self.pretasks = list(map(int, line[2].split(";")))
        except:
            self.pretasks = []

        self.children_finish = int(line[3])
        self.issue_type = issue_type(int(line[4]))
        self.other_issue = int(line[5])
        self.pbs = float(line[6])
        if self.issue_type == issue_type.INCOMING:
            self.reward = float(line[7])
            self.decom_penalty_rate = float(line[8])
        else:
            self.reward = 0
            self.decom_penalty_rate = 0
        self.est_o = int(line[9])
        self.dl_o = int(line[10])
        self.duration = int(line[11])

        self.est = max(self.est_o, tasks_est_o)
        self.dl = min(self.dl_o, tasks_dl_o)

        self.negotiation_success: int = -1
        # -1 for unknown, 0 for failure, 1 for success

    def __str__(self) -> str:
        return f"agent:{self.agent} parent:{self.parent} pretasks:{self.pretasks} children_finish:{self.children_finish} issue_type:{self.issue_type.name} other_issue:{self.other_issue} pbs:{self.pbs} r:{self.reward} p:{self.decom_penalty_rate}"


class agent:
    def __init__(self, nego_dl_o) -> None:
        self.issues: list[int] = []

        # issues that are either outgoing or incoming
        self.negotiation_issues: list[int] = []

        # 0 for free, 1 for occupied. Length is the number of timesteps
        self.occupied: list[int] = [0 for _ in range(nego_dl_o + 1)]

        self.assignment: tuple[tuple[int, int, int]] = ()
        self.ordering: tuple[int] = ()
        self.best_value: int = float("-inf")

    def __str__(self) -> str:
        return f"issues:{self.issues} negotiation_issues:{self.negotiation_issues} assignment:{self.assignment} ordering:{self.ordering} best_value:{self.best_value}"

### Load the data

In [279]:
input_file = os.path.join(os.getcwd(), "..", "data", "input.csv")

issue_graph: list[issue] = []
with open(input_file, "r") as f:
    num_agents, nego_est_o, nego_dl_o, tasks_est_o, tasks_dl_o = list(
        map(int, f.readline().split(","))
    )
    agents: list[agent] = [agent(nego_dl_o) for i in range(num_agents)]
    issue_num: int = 0
    for line in f:
        cur_issue = issue(issue_num, line, tasks_est_o, tasks_dl_o)
        issue_graph.append(cur_issue)
        agents[cur_issue.agent].issues.append(issue_num)
        issue_num += 1
    for i, issue_ in enumerate(issue_graph):
        # copying values from incoming issues to their corresponding outgoing issues
        if issue_.issue_type == issue_type.OUTGOING:
            issue_graph[i].decom_penalty_rate = issue_graph[
                issue_.other_issue
            ].decom_penalty_rate
            issue_graph[i].reward = issue_graph[issue_.other_issue].reward
            issue_graph[i].est_o = issue_graph[issue_.other_issue].est_o
            issue_graph[i].dl_o = issue_graph[issue_.other_issue].dl_o
            issue_graph[i].duration = issue_graph[issue_.other_issue].duration

print(f"{nego_est_o=}")
print(f"{nego_dl_o=}")
print(f"{num_agents=}")
print("Agents and their issues:")
for i, agent in enumerate(agents):
    print(f"Agent {i}: {agent.issues}")

print("\nIssues and their components:")
print(
    tabulate(
        [
            [
                i,
                issue_graph[i].agent,
                issue_graph[i].parent,
                issue_graph[i].pretasks,
                issue_graph[i].children_finish,
                issue_graph[i].issue_type.name,
                issue_graph[i].other_issue,
                issue_graph[i].pbs,
                issue_graph[i].reward,
                issue_graph[i].decom_penalty_rate,
                issue_graph[i].est_o,
                issue_graph[i].dl_o,
                issue_graph[i].duration,
            ]
            for i in range(len(issue_graph))
        ],
        headers=[
            "ID",
            "agent",
            "parent",
            "pretasks",
            "children_finish",
            "issue_type",
            "other_issue",
            "pbs",
            "reward",
            "decom_penalty_rate",
            "est_o",
            "dl_o",
            "duration",
        ],
    )
)

nego_est_o=0
nego_dl_o=10
num_agents=3
Agents and their issues:
Agent 0: [0, 1]
Agent 1: [2, 3, 4, 5, 6, 7, 8]
Agent 2: [9, 10, 11, 12]

Issues and their components:
  ID    agent    parent  pretasks      children_finish  issue_type      other_issue    pbs    reward    decom_penalty_rate    est_o    dl_o    duration
----  -------  --------  ----------  -----------------  ------------  -------------  -----  --------  --------------------  -------  ------  ----------
   0        0        -1  []                          0  OUTGOING                  2   0.98        10                   1          1      27           6
   1        0        -1  []                          0  OUTGOING                 10   0.89         9                   0.4        3      30           5
   2        1        -1  []                          0  INCOMING                  0   0.7         10                   1          1      27           6
   3        1         2  []                          0  LOCAL             

# Phase 1: Pre Negotiation

### Finding those issues that are going to be negotiated over

In [280]:
def find_negotiation_issues():
    print("Negotiation issues:")
    for i, agent in enumerate(agents):
        agent.negotiation_issues = [
            issueID
            for issueID in agent.issues
            if issue_graph[issueID].issue_type
            in [issue_type.OUTGOING, issue_type.INCOMING]
        ]
        print(f"Agent {i}: {agent.negotiation_issues}")


find_negotiation_issues()

Negotiation issues:
Agent 0: [0, 1]
Agent 1: [2, 6]
Agent 2: [9, 10]


### Complete search for the optimal (negotiation ordering, feature assignment tuple)

Helper functions to calculate flexibility, ps (probability of success), and EV (expected value):

In [281]:
def flexibility(est, dl, duration) -> float:
    """
    Returns the flexibility of the task in the range [0, 1]
    """
    return (dl - est - duration) / duration


def ps(pbs, est, dl, duration, c=0):
    """
    ps: probability of success
    """
    return 2 * pbs * (atan(flexibility(est, dl, duration))) / np.pi


def EV(
    assignment: tuple[tuple[int, int, int]],
    agent_ID: int,
) -> float:
    """
    Returns the estimated value of the assignment for the agent
    """
    ev: float = 0
    pbs_list = [
        issue_graph[issueID].pbs for issueID in agents[agent_ID].negotiation_issues
    ]
    ps_list = [
        ps(pbs, est, dl, dur) for (est, dl, dur), pbs in zip(assignment, pbs_list)
    ]
    issue_ID_list = [issueID for issueID in agents[agent_ID].negotiation_issues]
    for outcome in product([0, 1], repeat=len(assignment)):
        p = np.prod([ps if outcome[i] else 1 - ps for i, ps in enumerate(ps_list)])
        r = np.sum(
            [
                issue_graph[issueID].reward
                if outcome[i]
                else issue_graph[issueID].reward
                * issue_graph[issueID].decom_penalty_rate
                for i, issueID in zip(range(len(outcome)), issue_ID_list)
            ]
        )
        ev += p * r
    return ev


def is_valid_assignment(assignment) -> bool:
    """
    Returns true if the assignment is valid, false otherwise
    """
    for (est, dl, dur) in assignment:
        if est + dur > dl or est >= dl:
            return False

    def compare(a, b):
        # TODO: this may not be correct, how to find out if an assignment is valid?
        """
        Sorts by deadline first (lowest first), then est (lowest first), then duration (highest first)
        """
        if a[0] != b[0]:
            return a[0] - b[0]
        elif a[1] != b[1]:
            return a[1] - b[1]
        else:
            return b[2] - a[2]

    sorted_ass = list(assignment)
    sorted_ass.sort(key=cmp_to_key(compare))
    t = -1  # any negative number suffices since est >= 0 so t+1 >= 0
    for (est, dl, dur) in sorted_ass:
        t = max(t + 1, est) + dur
        if t > min(dl, nego_dl_o):
            return False

    return True


def find_ordering(assignment):
    """
    Returns a tuple of *indices* of the agents negotiation issues by inferring the ordering from the assignment
    """

    def compare(a, b):
        """
        Sorting by est and then deadline
        """
        return a[0] - b[0] if a[0] != b[0] else a[1] - b[1]

    return tuple((i[0] for i in sorted(enumerate(assignment), key=cmp_to_key(compare))))

The complete search itself:

In [282]:
def complete_search() -> None:
    """
    Finds the best assignment for each agent
    """
    # TODO: multithread this by making a thread for each agent
    for agent_ID, agent in enumerate(agents):
        best_value = float("-inf")
        best_assignment = None
        est_space = range(nego_est_o, nego_dl_o)
        dl_space = range(nego_est_o, nego_dl_o + 1)
        duration_space = range(1, nego_dl_o - nego_est_o)
        for assignment in product(
            product(est_space, dl_space, duration_space),
            repeat=len(agent.negotiation_issues),
        ):
            if is_valid_assignment(assignment):
                ev = EV(assignment, agent_ID)
                if ev > best_value:
                    best_value = ev
                    best_assignment = assignment
        ordering = find_ordering(assignment)
        agents[agent_ID].assignment = best_assignment
        agents[agent_ID].ordering = ordering
        agents[agent_ID].best_value = best_value

    print(
        tabulate(
            [
                (
                    agent_ID,
                    agent.negotiation_issues,
                    agent.best_value,
                    agent.assignment,
                    agent.ordering,
                )
                for (agent_ID, agent) in enumerate(agents)
            ],
            headers=[
                "Agent ID",
                "Negotiation Issues",
                "Best EV",
                "Assignment",
                "Ordering",
            ],
        )
    )


complete_search()

  Agent ID  Negotiation Issues      Best EV  Assignment               Ordering
----------  --------------------  ---------  -----------------------  ----------
         0  [0, 1]                  18.0674  ((0, 7, 6), (0, 10, 1))  (0, 1)
         1  [2, 6]                  29.5     ((0, 7, 4), (0, 1, 1))   (0, 1)
         2  [9, 10]                 27.969   ((0, 1, 1), (0, 10, 1))  (0, 1)


# Phase 2: Negotiations

In [283]:
# Following the earliest-finish-time policy
def run_negotiations() -> None:
    for agent_ID, agent in enumerate(agents):
        ordered_issues = [agent.negotiation_issues[i] for i in agent.ordering]
        ordered_assignment = [agent.assignment[i] for i in agent.ordering]
        for issue_ID in ordered_issues:
            if issue_graph[issue_ID].negotiation_success != -1:
                continue
            other_issue_ID = issue_graph[issue_ID].other_issue
            other_agent_ID = issue_graph[other_issue_ID].agent
            time_steps: list[int] = []

            nego_duration = ordered_assignment[ordered_issues.index(issue_ID)][2]
            for i in range(len(agent.occupied)):
                if len(time_steps) == nego_duration:
                    break
                if not agent.occupied[i] and not agents[other_agent_ID].occupied[i]:
                    time_steps.append(i)

            if len(time_steps) < nego_duration:
                # negotiation failed
                issue_graph[issue_ID].negotiation_success = 0
                issue_graph[other_issue_ID].negotiation_success = 0
                print(
                    f"Negotiation failed for agents {agent_ID} and {other_agent_ID} over issue {issue_ID}"
                )
                continue

            # negotiation has succeeded:
            print(
                f"Negotiation succeeded for agents {agent_ID} and {other_agent_ID} over issue {issue_ID}. Time steps: {time_steps} (duration: {nego_duration})"
            )
            issue_graph[issue_ID].negotiation_success = 1
            issue_graph[other_issue_ID].negotiation_success = 1

    # set the features of the issue execution for phase 3
    for issue_ID, issue in enumerate(issue_graph):
        if (
            issue.issue_type in [issue_type.INCOMING, issue_type.OUTGOING]
            and issue.other_issue > issue_ID
        ):
            free_time = (
                issue_graph[issue_ID].dl_o
                - issue_graph[issue_ID].est_o
                - issue_graph[issue_ID].duration
            )
            issue_graph[issue_ID].est = issue_graph[issue_ID].est_o + random.randint(0, int(free_time / 2))
            issue_graph[issue_ID].dl = issue_graph[issue_ID].dl_o - random.randint(0, free_time - int(free_time / 2))
            issue_graph[issue.other_issue].est = issue_graph[issue_ID].est
            issue_graph[issue.other_issue].dl = issue_graph[issue_ID].dl

    # tabulate issue, negotiation success, est, dl, duration
    negotiation_success_in_words = {1: "Success", 0: "Failure", -1: "NA"}
    print(
        tabulate(
            [
                (
                    issue_ID,
                    issue.negotiation_success,
                    issue.est,
                    issue.dl,
                    issue.duration,
                )
                for issue_ID, issue in enumerate(issue_graph)
            ],
            headers=["Issue ID", "Negotiation Success", "est", "dl", "duration"],
        )
    )


run_negotiations()

Negotiation succeeded for agents 0 and 1 over issue 0. Time steps: [0, 1, 2, 3, 4, 5] (duration: 6)
Negotiation succeeded for agents 0 and 2 over issue 1. Time steps: [0] (duration: 1)
Negotiation succeeded for agents 1 and 2 over issue 6. Time steps: [0] (duration: 1)
  Issue ID    Negotiation Success    est    dl    duration
----------  ---------------------  -----  ----  ----------
         0                      1     11    17           6
         1                      1     14    19           5
         2                      1     11    17           6
         3                     -1      1    26           2
         4                     -1      1    25           2
         5                     -1      2    30           3
         6                      1     12    17           5
         7                     -1      2    27           3
         8                     -1      9    30           2
         9                      1     12    17           5
        10            

# Phase 3: Scheduling Tasks